### Install Libraries

In [ ]:
!pip install qiskit
!pip install qiskit_algorithms
!pip install qiskit_ibm_runtime
!pip install nevergrad
!pip install pyDOE
!pip install scikit-quant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requir

### Hamiltonian Construction

In [ ]:
import numpy as np
import itertools
from qiskit.quantum_info import SparsePauliOp
import qiskit
import random
import qiskit_algorithms
import scipy as sp

In [ ]:
def construct_hamiltonian(j1, j2, grid):
  def nearest_neighbor(grid, i, j):
    i, j = i % len(grid[0]), j % len(grid)
    look_at = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    result = []
    for element in look_at:
      dx, dy = element
      result.append([(i + dx) % len(grid[0]), (j + dy) % len(grid)])
    return result

  def next_nearest_neighbor(grid, i, j):
    look_at = [[1, 1], [1, -1], [-1, 1], [-1, -1]]
    result = []
    for element in look_at:
      dx, dy = element
      result.append([(i + dx) % len(grid[0]), (j + dy) % len(grid)])
    return result

  def generate_dot_product(grid, term, idxA, idxB):
    operation_template = ['I' for element in range(len(grid[0]) * len(grid))]
    dot_product = SparsePauliOp(('I' * len(grid[0]) * len(grid)), coeffs=[0])
    for direction in ['X', 'Y', 'Z']:
      operation = operation_template
      operation[idxA], operation[idxB] = direction, direction
      dot_product += SparsePauliOp("".join(operation), coeffs=[term])
    return dot_product


  hamilonian = SparsePauliOp(('I' * len(grid[0]) * len(grid)), coeffs=[0])
  for i in range(len(grid[0])):
    for j in range(len(grid)):
      n_neighbors = nearest_neighbor(grid, i, j)
      nn_neighbors = next_nearest_neighbor(grid, i, j)

      for neighbor in n_neighbors:
        idxA = (j * len(grid)) + i
        idxB = (neighbor[1] * len(grid)) + neighbor[0]
        hamilonian += generate_dot_product(grid, j1, idxA, idxB)
      for neighbor in nn_neighbors:
        idxA = (j * len(grid)) + i
        idxB = (neighbor[1] * len(grid)) + neighbor[0]
        hamilonian += generate_dot_product(grid, j2, idxA, idxB)

  return hamilonian.simplify()

In [ ]:
H = construct_hamiltonian(1.0, 0.5, [[1/2 for i in range(2)] for j in range(2)]).simplify()

### Let us have something to compare to, compute it manually

## Implementation: Lanzcos Basis

In [ ]:
sparse_H = H.to_matrix(sparse=True)

In [ ]:
import scipy

In [ ]:
v = np.array([1 if bin(i)[2:] == "1010" else 0 for i in range(2**4)])
v_csc = scipy.sparse.csc_matrix(v).T

In [ ]:
krylov_vectors = [v_csc]
krylov_dim = 50
temp = v_csc
for k in range(1, krylov_dim):
  temp = k * sparse_H @ temp
  krylov_vectors.append(temp)
  print("Making Krylov Vector: ", k)

Making Krylov Vector:  1
Making Krylov Vector:  2
Making Krylov Vector:  3
Making Krylov Vector:  4
Making Krylov Vector:  5
Making Krylov Vector:  6
Making Krylov Vector:  7
Making Krylov Vector:  8
Making Krylov Vector:  9
Making Krylov Vector:  10
Making Krylov Vector:  11
Making Krylov Vector:  12
Making Krylov Vector:  13
Making Krylov Vector:  14
Making Krylov Vector:  15
Making Krylov Vector:  16
Making Krylov Vector:  17
Making Krylov Vector:  18
Making Krylov Vector:  19
Making Krylov Vector:  20
Making Krylov Vector:  21
Making Krylov Vector:  22
Making Krylov Vector:  23
Making Krylov Vector:  24
Making Krylov Vector:  25
Making Krylov Vector:  26
Making Krylov Vector:  27
Making Krylov Vector:  28
Making Krylov Vector:  29
Making Krylov Vector:  30
Making Krylov Vector:  31
Making Krylov Vector:  32
Making Krylov Vector:  33
Making Krylov Vector:  34
Making Krylov Vector:  35
Making Krylov Vector:  36
Making Krylov Vector:  37
Making Krylov Vector:  38
Making Krylov Vector:

In [ ]:
def orthonormalize_and_stack(vectors):
    """
    Orthonormalizes a list of sparse or dense vectors and stacks them into a matrix.

    Parameters:
        vectors (list): A list of vectors (np.ndarray or similar types that can be converted to arrays).

    Returns:
        np.ndarray: A matrix with orthonormal columns formed from the input vectors.
    """
    # Initialize an empty list to store the orthonormal vectors
    orthonormal_vectors = []

    for v in vectors:
        # Convert sparse vector to dense array if necessary
        if hasattr(v, 'toarray'):
            v = v.toarray().flatten()
        else:
            v = np.array(v).flatten()

        # Orthogonalize against all previous vectors
        for u in orthonormal_vectors:
            v -= np.dot(u, v) * u

        # Normalize the vector
        norm_v = np.linalg.norm(v)
        if norm_v > 0:  # Avoid division by zero
            v_normalized = v / norm_v
            orthonormal_vectors.append(v_normalized)

    # Stack the orthonormal vectors horizontally to form the matrix
    K_matrix = np.hstack([v.toarray().flatten()[:, np.newaxis] for v in vectors])

    return K_matrix

In [ ]:
K_matrix = orthonormalize_and_stack(krylov_vectors)

In [ ]:
HK_matrix = sparse_H.dot(K_matrix)

In [ ]:
print(HK_matrix)

[[ 0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0.00000000e+000+0.j  0.00000000e+000+0.j
   0.00000000e+000+0.j  0

In [ ]:
H_matrix_krylov = np.zeros((len(krylov_vectors), len(krylov_vectors)), dtype=complex)
# Compute the upper triangular part of the matrix
for i in range(krylov_dim):
  for j in range(krylov_dim):
    element = K_matrix[:, i].conj().T.dot(HK_matrix[:, j])
    H_matrix_krylov[i, j] = element

In [ ]:
print(H_matrix_krylov)

[[-8.00000000e+000+0.j  3.20000000e+002+0.j -5.12000000e+003+0.j ...
   2.30026122e+125+0.j -8.83300307e+127+0.j  3.11628348e+131+0.j]
 [ 3.20000000e+002+0.j -2.56000000e+003+0.j  3.35872000e+005+0.j ...
  -1.84020897e+126+0.j  6.35976221e+129+0.j -2.49302679e+132+0.j]
 [-5.12000000e+003+0.j  3.35872000e+005+0.j -5.37395200e+006+0.j ...
   2.64990092e+128+0.j -1.01756195e+131+0.j  3.58995857e+134+0.j]
 ...
 [ 2.30026122e+125+0.j -1.84020897e+126+0.j  2.64990092e+128+0.j ...
  -1.46977824e+249+0.j  5.07955360e+252+0.j -1.99118501e+255+0.j]
 [-8.83300307e+127+0.j  6.35976221e+129+0.j -1.01756195e+131+0.j ...
   5.07955360e+252+0.j -1.95054858e+255+0.j  6.88153540e+258+0.j]
 [ 3.11628348e+131+0.j -2.49302679e+132+0.j  3.58995857e+134+0.j ...
  -1.99118501e+255+0.j  6.88153540e+258+0.j -2.69756188e+261+0.j]]


In [ ]:
S_matrix_krylov = (K_matrix.conj().T).dot(K_matrix)

### Let us make sure the basis serves this system well

In [ ]:
from scipy.linalg import eigh

gnd_en_circ_est_list = []
gnd_en_circ_est = 1000
last_one = None
for d in range(1, len(krylov_vectors)):
  # Solve generalized eigenvalue problem using scipy
  try:
    S_matrix_krylov[:d, :d] = (S_matrix_krylov[:d, :d] + S_matrix_krylov[:d, :d].T)/2
    eigvals, eigvecs = eigh(H_matrix_krylov[:d, :d], S_matrix_krylov[:d, :d])
    gnd_en_circ_est = eigvals[0]
    gnd_en_circ_est_list.append(gnd_en_circ_est)
    print('Iteration: The estimated ground state energy is: ', gnd_en_circ_est, " Krylov Dim: ", d)
    last_one = eigvecs[:, 0]
    print(eigvals)
  except:
    print("Failed, not positive definite")
    pass

Iteration: The estimated ground state energy is:  -8.0  Krylov Dim:  1
[-8.]
Iteration: The estimated ground state energy is:  -17.88854381999832  Krylov Dim:  2
[-17.88854382  17.88854382]
Iteration: The estimated ground state energy is:  -23.999999999999996  Krylov Dim:  3
[-24.  -8.  24.]
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite
Failed, not positive definite


In [ ]:
print(eigh(sparse_H.toarray())[0])

[-24. -24.  -8.  -8.  -8.  -8.  -8.  -8.  -8.  -8.  -8.  24.  24.  24.
  24.  24.]


### Perform Hamiltonian Encoding

In [ ]:
def generate_normalized_H(H):
  H = H.simplify()
  coeffs = H.coeffs
  H.coeffs = np.array([np.abs(el) for el in coeffs]) / np.linalg.norm([np.abs(el) for el in coeffs])
  return H

In [ ]:
import math
H_eff = generate_normalized_H(H)
block_terms = int(math.ceil(np.log2(H_eff.coeffs.shape[0])))
print(block_terms)
H_eff_sparse = H_eff.to_matrix(sparse=True)

5


In [ ]:
def generate_U(ham, qubits, block_terms):
  operator = None
  idx = 0
  U = scipy.sparse.csr_matrix((2**(qubits + block_terms), 2**(qubits + block_terms)))

  for coeff, term in zip(ham.coeffs, ham.paulis):
    print("On term: ", idx+1, " out of ", len(ham.coeffs))
    binary_term = str(bin(idx)[2:]).zfill(block_terms)
    v_pauli = np.array([1 if bin(i)[2:].zfill(block_terms) == bin(idx)[2:].zfill(block_terms) else 0 for i in range(2**block_terms)])
    idx += 1

    v_pauli_sparse = scipy.sparse.csr_matrix(v_pauli)
    v_pauli_sparse = (v_pauli_sparse.conj().T @ v_pauli_sparse)
    term = term.to_matrix(sparse=True)
    U += scipy.sparse.kron(v_pauli_sparse, term, format='csr')

  return U

In [ ]:
U = generate_U(H.simplify(), H.num_qubits, block_terms)

On term:  1  out of  18
On term:  2  out of  18
On term:  3  out of  18
On term:  4  out of  18
On term:  5  out of  18
On term:  6  out of  18
On term:  7  out of  18
On term:  8  out of  18
On term:  9  out of  18
On term:  10  out of  18
On term:  11  out of  18
On term:  12  out of  18
On term:  13  out of  18
On term:  14  out of  18
On term:  15  out of  18
On term:  16  out of  18
On term:  17  out of  18
On term:  18  out of  18


In [ ]:
def nearest_unitary(A):
    # Perform singular value decomposition
    U, s, Vh = np.linalg.svd(A)

    # Construct the nearest unitary matrix
    U_nearest = np.dot(U, Vh)

    return U_nearest

U = nearest_unitary(U.toarray())

### Generate G matrix from Householder method

In [ ]:
import numpy as np

from scipy.linalg import qr
def create_householder_matrix(coeffs, num_ancillary):
    # Calculate square roots of normalized coefficients
    coeffs = np.array(coeffs)
    normalized_coeffs = np.sqrt(coeffs / np.sum(coeffs))

    # Determine the dimension of the space
    dim = 2**num_ancillary

    # Initial state |0>
    x = np.zeros(dim)
    x[0] = 1

    # Target state |G>
    b = np.zeros(dim)
    b[:len(normalized_coeffs)] = normalized_coeffs

    # Compute the Householder vector v
    v = x - b
    v /= np.linalg.norm(v)  # Normalize v to ensure the reflector is unitary

    # Householder transformation matrix
    H = np.eye(dim) - 2 * np.outer(v, v.conj())

    return H

def create_superposition_state(coeffs, num_ancillary, num_total):
    q = create_householder_matrix(coeffs, num_ancillary)
    # Construct the full unitary including the identity on the remaining qubits
    identity_size = 2**(num_total - num_ancillary)  # Size of the identity matrix for the rest of the qubits
    full_unitary = scipy.sparse.kron(scipy.sparse.csc_matrix(q.conj().T), scipy.sparse.eye(identity_size))
    full_unitary_orign = scipy.sparse.kron(scipy.sparse.csc_matrix(q), scipy.sparse.eye(identity_size))
    return full_unitary_orign, full_unitary, q, q.conj().T

### Prepare Statevector equivelent

In [ ]:
# Apply G's definition: superposition of Pauli subspace indicies, |G> = sum_i^T sqrt(alpha_i)|i>
def construct_G_vec(ham):
    # Calculate the normalized coefficients
    coeffs = np.array([np.absolute(element) for element in ham.coeffs])
    coeffs /= np.sum(coeffs)

    # Square root of the normalized coefficients
    sqrt_coeffs = np.sqrt(coeffs)

    # Create the state vector directly as a sparse matrix
    # The state vector will be indexed properly since range(len(sqrt_coeffs)) creates the correct indices
    indices = np.arange(len(sqrt_coeffs))
    v = scipy.sparse.csr_matrix((sqrt_coeffs, (indices, np.zeros_like(indices))), shape=(2**(5), 1))

    return v

In [ ]:
G, G_inverse, q, qT = create_superposition_state(H_eff.simplify().coeffs, 5, 5+4)

<ipython-input-22-af231a384850>:18: ComplexWarning: Casting complex values to real discards the imaginary part
  b[:len(normalized_coeffs)] = normalized_coeffs


In [ ]:
print(G_inverse)

  (0, 0)	0.2357022603955159
  (1, 1)	0.2357022603955159
  (2, 2)	0.2357022603955159
  (3, 3)	0.2357022603955159
  (4, 4)	0.2357022603955159
  (5, 5)	0.2357022603955159
  (6, 6)	0.2357022603955159
  (7, 7)	0.2357022603955159
  (8, 8)	0.2357022603955159
  (9, 9)	0.2357022603955159
  (10, 10)	0.2357022603955159
  (11, 11)	0.2357022603955159
  (12, 12)	0.2357022603955159
  (13, 13)	0.2357022603955159
  (14, 14)	0.2357022603955159
  (15, 15)	0.2357022603955159
  (16, 0)	0.23570226039551584
  (17, 1)	0.23570226039551584
  (18, 2)	0.23570226039551584
  (19, 3)	0.23570226039551584
  (20, 4)	0.23570226039551584
  (21, 5)	0.23570226039551584
  (22, 6)	0.23570226039551584
  (23, 7)	0.23570226039551584
  (24, 8)	0.23570226039551584
  :	:
  (487, 487)	1.0
  (488, 488)	1.0
  (489, 489)	1.0
  (490, 490)	1.0
  (491, 491)	1.0
  (492, 492)	1.0
  (493, 493)	1.0
  (494, 494)	1.0
  (495, 495)	1.0
  (496, 496)	1.0
  (497, 497)	1.0
  (498, 498)	1.0
  (499, 499)	1.0
  (500, 500)	1.0
  (501, 501)	1.0
  (502, 5

### Some Sanity Checks

In [ ]:
import numpy as np

def is_unitary(m):
    # Ensure matrix multiplication is used, not element-wise multiplication
    return np.allclose(np.eye(m.shape[0]), m.conj().T @ m)

# Assuming 'q' and 'qT' (q.conj().T) are the matrices you're testing
print(is_unitary(q))   # Check if q is unitary
print(is_unitary(qT))  # Check if qT (q.conj().T) is unitary


True
True


In [ ]:
G_state_ancillary = np.zeros((2**5,1))
G_state_ancillary[0] = 1  # This is the |0>_a state
print(G.shape)
print(G_state_ancillary.shape)
G_ancillary = q @ G_state_ancillary
print(G_ancillary.shape)

(512, 512)
(32, 1)
(32, 1)


In [ ]:
R = scipy.sparse.kron(2 * np.outer(G_ancillary.T.conj(), G_ancillary) - scipy.sparse.eye(2**5), scipy.sparse.eye(2**4))

In [ ]:
r = 2 * np.outer(G_ancillary.T.conj(), G_ancillary) - scipy.sparse.eye(2**5)

In [ ]:
is_unitary((2 * np.outer(G_ancillary.T.conj(), G_ancillary) - scipy.sparse.eye(2**5)))

True

In [ ]:
print(R.shape)
print(G.shape)
print(G_inverse.shape)
print(U.shape)

(512, 512)
(512, 512)
(512, 512)
(512, 512)


### Now, preform necessary measurements

In [ ]:
import copy

def create_projection_operator(i, num_ancillary):
    """Create a projection operator for the i-th state in the ancillary register."""
    dim_ancillary = 2**num_ancillary
    # Create a matrix with a single 1 at position (i, i)
    A_i = scipy.sparse.csc_matrix(([1], ([i], [i])), shape=(dim_ancillary, dim_ancillary))
    return A_i

def measure_in_basis(state, ham, num_ancillary):
    """Measure the expectation value conditioned on the ancillary qubits being in state i."""
    expectation = 0.0
    i = 0
    for coeff, term in zip(ham.coeffs, ham.paulis):
        A_i = create_projection_operator(i, num_ancillary)
        pauli_matrix = term.to_matrix(sparse=True)
        M = sp.sparse.kron(A_i, pauli_matrix)  # Measurement operator
        expectation += coeff * (state.conj().T @ M @ state)[0,0]  # Compute the expectation value
        i += 1
    return expectation

def create_circuit(k, ham):
  global R, G, G_inverse, U

  inital_ancillary = construct_G_vec(ham)
  inital_psi = np.array([1 if bin(i)[2:].zfill(4) == "1010" else 0 for i in range(2**4)])
  inital_psi_sparse = scipy.sparse.csc_matrix(inital_psi)
  inital = scipy.sparse.kron(inital_ancillary, inital_psi_sparse.T)

  if(k % 2 == 0):
    for _ in range(int(k/2)):
      inital = U @ inital
      inital = R @ inital
    inital = G_inverse @ inital

    A_0 = create_projection_operator(0, 5)
    identity = sp.sparse.eye(2**5, format='csc')
    measurement_operator = 2 * A_0 - identity
    full_operator = sp.sparse.kron(measurement_operator, sp.sparse.eye(2**(4), format='csc'))
    probability = (inital.conj().T @ full_operator @ inital)[0, 0]
    return 1 if abs(abs(probability) - 1) <= 0.1 else -1
  else:
    for _ in range(int(math.floor(k/2))):
      inital = U @ inital
      inital = R @ inital
    return measure_in_basis(inital, ham, 5)

In [ ]:
last_sign = 1
krylov_entries = []
krylov_vectors = [0 for i in range(15)]
for d in range(2*len(krylov_vectors)):
  entry = create_circuit(d, H_eff)
  if(d % 2 == 1):
    print(entry)
  krylov_entries.append(entry)

(-0.026189140043946207+0j)
(0.07210096580000006+0j)
(-0.10123194340108516+0j)
(0.10997205472090098+0j)
(-0.10255901931056735+0j)
(0.08879376652548326+0j)
(-0.07936317077720824+0j)
(0.08086470071919627+0j)
(-0.09275539156055272+0j)
(0.10758702043729701+0j)
(-0.11442326577802944+0j)
(0.1039148372129519+0j)
(-0.07276636803981508+0j)
(0.02559439446925603+0j)
(0.026668772672951077+0j)


In [ ]:
print(krylov_entries[::2])

[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1]


### Annoying postprocessing of observables

In [ ]:
def generate_subspace_matricies(krylov_entries, krylov_dim):
  H = np.zeros((krylov_dim, krylov_dim))
  S = np.zeros((krylov_dim, krylov_dim))

  for i in range(krylov_dim):
    for j in range(krylov_dim):
      S[i, j] = 0.5 * (krylov_entries[i+j] + krylov_entries[abs(i - j)])
  for i in range(krylov_dim):
    for j in range(krylov_dim):
      H[i, j] = 0.25 * (krylov_entries[i+j+1] + krylov_entries[abs(i + j - 1)] +
                        krylov_entries[abs(i-j+1)] + krylov_entries[abs(i - j - 1)])
  return H, S

In [ ]:
import numpy as np
from scipy.linalg import eigh, cholesky
from itertools import combinations


def solve_eigen_problem(H_subspace, S_subspace):
    """ Solve the generalized eigenvalue problem and return the smallest eigenvalue and its corresponding eigenvector. """
    eigenvals = None
    for d in range(H_subspace.shape[0]):
        try:
            eigvals, eigvecs = eigh(H_subspace[:d, :d], S_subspace[:d, :d] + 0.755e-3 * np.eye(d))
            print(d, eigvals)
            min_eigval_index = np.argmin(eigvals)
            eigenvals = eigvals[min_eigval_index]
        except:
            pass
    return eigenvals

In [ ]:
H_subspace, S_subspace = generate_subspace_matricies(krylov_entries, len(krylov_entries)//2)

<ipython-input-200-b6387eaddf97>:7: ComplexWarning: Casting complex values to real discards the imaginary part
  S[i, j] = 0.5 * (krylov_entries[i+j] + krylov_entries[abs(i - j)])
<ipython-input-200-b6387eaddf97>:10: ComplexWarning: Casting complex values to real discards the imaginary part
  H[i, j] = 0.25 * (krylov_entries[i+j+1] + krylov_entries[abs(i + j - 1)] +


In [ ]:
print(H_subspace)
print(S_subspace)

[[-2.61891400e-02  0.00000000e+00  2.29559129e-02 -1.00000000e+00
  -1.45654888e-02 -1.00000000e+00  4.37005566e-03 -1.00000000e+00
   3.70651771e-03 -1.00000000e+00 -6.88262639e-03 -1.00000000e+00
   4.71529787e-03 -1.00000000e+00  7.50764971e-04]
 [ 0.00000000e+00 -1.61661358e-03 -5.00000000e-01  4.19521204e-03
  -1.00000000e+00 -5.09771657e-03 -1.00000000e+00  4.03828668e-03
  -1.00000000e+00 -1.58805434e-03 -1.00000000e+00 -1.08366426e-03
  -1.00000000e+00  2.73303142e-03 -1.00000000e+00]
 [ 2.29559129e-02 -5.00000000e-01 -2.03773144e-02 -5.00000000e-01
   1.36629843e-02 -1.00000000e+00 -5.42948555e-03 -1.00000000e+00
  -1.25628537e-03 -1.00000000e+00  4.21090779e-03 -1.00000000e+00
  -3.06593071e-03 -1.00000000e+00 -6.15023773e-04]
 [-1.00000000e+00  4.19521204e-03 -5.00000000e-01 -1.09095422e-02
  -5.00000000e-01  1.33312153e-02 -1.00000000e+00 -1.07240576e-02
  -1.00000000e+00  4.54267677e-03 -1.00000000e+00  2.22864134e-03
  -1.00000000e+00 -6.41398591e-03 -1.00000000e+00]
 [-1

In [ ]:
solve_eigen_problem(H_subspace, S_subspace)

1 [-0.02616938]
2 [-23.46954357  -0.02588192]


-23.46954356792272

### Seems like we are ready to put it on a Quantum Computer

#### Question: How do we make the block encoding efficiently? Luckily, a library called Fable can do this for us!

In [ ]:
!pip install fable-circuits

In [ ]:
from fable import fable

In [ ]:
print(U.shape)

(512, 512)


In [ ]:
circ, alpha = fable(H.to_matrix(), 1e-2)
circ = pm.run(circ)
print(circ.depth())


1092


#### I am going to accept a 1% loss of fidelity in exchange for a much shorter circuit depth


In [ ]:
R_QC = R.toarray()
G_QC = G.toarray()
G_inverse_QC = G_inverse.toarray()
U_QC = U
circ, alpha = fable(H.to_matrix(), 1e-2)

In [ ]:
print(circ.depth())
print(alpha)

180
24.00000001490116


#### Fable, of course, does query register after coefficient register, unlike Exact and Efficient Lanczos Method, so let us fix that

In [ ]:
print(circ.num_qubits)
circ = circ.reverse_bits()

9


In [ ]:
import numpy as np
from scipy import sparse
import qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, Pauli
from qiskit.circuit.library import UnitaryGate
from qiskit.primitives import Estimator

#### Well, 180 gates just for each query with 15 queries necessary...uh no, and we have to accept an error of O(1) to get to a <50 circuit depth! So, we need a way to prevent the linear scaling to run on NISQ devices

#### Let us acccept an approximate Lanczos basis construction and variationally find an approximate diagonalized formulation of the query operator to make the O(i) [i - index of Lanczos vector] into a O(1) by just modifying the coefficients in the diagonal matrix!

In [ ]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.circuit import QuantumCircuit
from qiskit.synthesis import LieTrotter
from qiskit.circuit import ParameterVector, Parameter
import random

class VariationalTimeEvolution(QuantumCircuit):
    def __init__(self, num_qubits_1, reps, n, real=True):
        super().__init__(num_qubits_1)
        self.num_qubits_1 = num_qubits_1
        self.reps = reps+1
        self.params = []

        params_theta_inverse = []
        params_phi_inverse = []

        gammas = []
        gamma = Parameter(f'gamma')
        for i in range(1, self.num_qubits_1):
          gammas.append(gamma)
        self.params.append(gamma)

        priors = []
        for rep in range(reps):
            # Even Terms
            for i in range(1,self.num_qubits_1-1,2):
                param_theta = Parameter(f'phi_W_{rep}_{i}_even')
                param_phi = Parameter(f'theta_W_{rep}_{i}_even')
                self.cx(i, i+1)
                self.append(qiskit.circuit.library.RGate(-param_theta, -param_phi), [i+1])
                self.cx(i+1, i)
                self.append(qiskit.circuit.library.RGate(param_theta, param_phi), [i+1])
                self.cx(i, i+1)
                self.params.append(param_theta)
                self.params.append(param_phi)
                params_theta_inverse.append(param_theta)
                params_phi_inverse.append(param_phi)

            # Odd Terms
            for i in range(2,self.num_qubits_1-1,2):
                param_theta = Parameter(f'phi_W_{rep}_{i}_odd')
                param_phi = Parameter(f'theta_W_{rep}_{i}_odd')
                self.cx(i, i+1)
                self.append(qiskit.circuit.library.RGate(-param_theta, -param_phi), [i+1])
                self.cx(i+1, i)
                self.append(qiskit.circuit.library.RGate(param_theta, param_phi), [i+1])
                self.cx(i, i+1)
                self.params.append(param_theta)
                self.params.append(param_phi)
                params_theta_inverse.append(param_theta)
                params_phi_inverse.append(param_phi)

            # Apply control Z
            for i in range(1,self.num_qubits_1-1):
                self.crz(n * gammas[i], 0, i)

            # Apply e^(-i/2 n ZZ)
            for i in range(1,self.num_qubits_1-1,2):
                self.cz(i, i+1)
                self.crz(n * gammas[i], 0, i+1)
                self.cz(i, i+1)

            # Apply e^(-i/2 n ZZ)
            for i in range(2,self.num_qubits_1-1,2):
                self.cz(i, i+1)
                self.crz(n * gammas[i], 0, i+1)
                self.cz(i, i+1)

            # Loop over repetitions in reverse order
            index = 0  # Start at the last set of parameters used
            priors = priors[::-1]
            params_theta_inverse = params_theta_inverse[::-1]
            params_phi_inverse = params_phi_inverse[::-1]

            # Odd Terms (Adjoint in reverse order)
            index = 0
            for i in reversed(range(2, self.num_qubits_1-1, 2)):
                param_theta = params_theta_inverse[index]
                param_phi = params_phi_inverse[index]
                index += 1
                self.cx(i, i+1)
                self.append(qiskit.circuit.library.RGate(param_theta, param_phi), [i+1])
                self.cx(i+1, i)
                self.append(qiskit.circuit.library.RGate(-param_theta, -param_phi), [i+1])
                self.cx(i, i+1)

            # Even Terms (Adjoint in reverse order)
            for i in reversed(range(1, self.num_qubits_1-1, 2)):
                param_theta = params_theta_inverse[index]
                param_phi = params_phi_inverse[index]
                index += 1
                self.cx(i, i+1)
                self.append(qiskit.circuit.library.RGate(param_theta, param_phi), [i+1])
                self.cx(i+1, i)
                self.append(qiskit.circuit.library.RGate(-param_theta, -param_phi), [i+1])
                self.cx(i, i+1)
        np.random.shuffle(self.params)

In [ ]:
def create_projection_operator(i, num_ancillary):
    """Create a projection operator for the i-th state in the ancillary register."""
    dim_ancillary = 2**num_ancillary
    data = np.zeros((dim_ancillary, dim_ancillary))
    data[i, i] = 1
    return Operator(data)

In [ ]:
def create_measurement_operator(num_ancillary):
    A_0 = create_projection_operator(0, num_ancillary)
    identity = np.eye(2**num_ancillary)
    measurement_operator = 2 * A_0.data - identity  # Create dense matrix for Qiskit Operator
    full_operator = Operator(measurement_operator)  # Convert to Qiskit Operator
    return full_operator

In [ ]:
def create_projection_operator_pauli(i, num_ancillary):
    """Create a projection operator for the i-th state in the ancillary register."""
    # Constructs a binary string where only the i-th ancillary qubit is 1 (rest are 0)
    # assuming ordering from most to least significant qubit
    binary_state = str(bin(i)[2:].zfill(5))
    label = "".join(["I" if binary_state[j] != "1" else "Z" for j in range(num_ancillary)])
    return SparsePauliOp.from_list([(label, 1)])


def measure_in_basis(qc, ham, num_ancillary):
    """Measure the expectation value conditioned on the ancillary qubits being in state i."""
    expectation = 0.0
    estimator = qiskit.primitives.Estimator()

    circuits = []
    observables = []
    for i, (coeff, term) in enumerate(zip(ham.coeffs, ham.paulis)):
        A_i = create_projection_operator_pauli(i, num_ancillary)
        pauli_matrix = SparsePauliOp(term)
        # Measurement operator combining ancillary projection with system Pauli
        M = A_i.tensor(pauli_matrix)
        observables.append(M)
        circuits.append(qc)

    results = estimator.run(circuits, observables).result()
    expectation = 0.0
    for i, (coeff, result) in enumerate(zip(ham.coeffs, results.values)):
        expectation += coeff * result

    return expectation

def measure_and_decide(qc, num_ancillary):
    measurement_operator = create_measurement_operator(num_ancillary)
    sampler = qiskit.primitives.Sampler()
    for i in range(4,5+4):
      qc.measure(i,i-4)
    result = sampler.run(circuits=[qc]).result()
    return 1 if abs(result.quasi_dists[0][0] - 1) <= 0.1 else -1

In [ ]:
def create_circuit(k, ham):
    global R_QC, G_QC, G_inverse_QC, U_QC, r, qT

    inital_ancillary = construct_G_vec(ham)
    inital_psi = np.array([1 if bin(i)[2:].zfill(4) == "1010" else 0 for i in range(2**4)])
    inital_psi_sparse = scipy.sparse.csc_matrix(inital_psi)
    inital = scipy.sparse.kron(inital_ancillary, inital_psi_sparse.T)


    # Create the quantum circuit
    num_qubits = int(np.log2(inital.shape[0]))  # Determine the number of qubits from the state vector size
    if(k % 2 == 0):
      qc = QuantumCircuit(num_qubits, 5)
    else:
      qc = QuantumCircuit(num_qubits)

    # Prepare the initial state
    initial_state = Statevector(inital.toarray())
    qc.append(qiskit.circuit.library.StatePreparation(initial_state), qc.qubits)

    # Applying unitary gates based on the condition of k
    if k % 2 == 0:
        for _ in range(int(k / 2)):
            qc.append(UnitaryGate(U_QC), range(9))
            qc.append(UnitaryGate(r), range(5))
        qc_copy = copy.deepcopy(qc)
        qc.append(UnitaryGate(qT), range(5))
        decision = measure_and_decide(qc, 5)
        print(decision)
        return decision, qc_copy
    else:
        for _ in range(int(k // 2)):
            qc.append(UnitaryGate(U_QC), range(9))
            qc.append(UnitaryGate(r), range(5))
        qc_copy = copy.deepcopy(qc)
        estimate= measure_in_basis(qc, ham, 5)
        print(estimate)
        return estimate, qc_copy

In [ ]:
last_sign = 1
krylov_entries_simulation = []
qc_copies = []
for d in range(0,2*len(krylov_vectors)):
  estimate, qc = create_circuit(d, H_eff)
  krylov_entries_simulation.append(estimate)
  qc_copies.append(qc)

-1
(-0.07856742013183882+0j)
-1
(-0.017782749412555976+0j)
-1
(-0.11064023529006296+0j)
-1
(-0.040814552701122796+0j)
-1
(-0.08918522471672245+0j)
-1
(-0.018114520402866967+0j)
-1
(-0.11201916547407784+0j)
-1
(-0.03632862689890268+0j)
-1
(-0.07626859815866549+0j)
-1
(-0.006786389585980148+0j)
-1
(-0.1031379538523801+0j)
-1
(-0.034716675034388994+0j)
-1
(-0.07043878775929481+0j)
-1
(-0.009350283528066812+0j)
-1
(-0.1121368325705272+0j)


In [ ]:
print(krylov_entries_simulation)

[-1, (-0.07856742013183882+0j), -1, (-0.017782749412555976+0j), -1, (-0.11064023529006296+0j), -1, (-0.040814552701122796+0j), -1, (-0.08918522471672245+0j), -1, (-0.018114520402866967+0j), -1, (-0.11201916547407784+0j), -1, (-0.03632862689890268+0j), -1, (-0.07626859815866549+0j), -1, (-0.006786389585980148+0j), -1, (-0.1031379538523801+0j), -1, (-0.034716675034388994+0j), -1, (-0.07043878775929481+0j), -1, (-0.009350283528066812+0j), -1, (-0.1121368325705272+0j)]


In [ ]:
H_subspace, S_subspace = generate_subspace_matricies(krylov_entries_simulation, 4)

<ipython-input-200-b6387eaddf97>:7: ComplexWarning: Casting complex values to real discards the imaginary part
  S[i, j] = 0.5 * (krylov_entries[i+j] + krylov_entries[abs(i - j)])
<ipython-input-200-b6387eaddf97>:10: ComplexWarning: Casting complex values to real discards the imaginary part
  H[i, j] = 0.25 * (krylov_entries[i+j+1] + krylov_entries[abs(i + j - 1)] +


In [ ]:
print(H_subspace, S_subspace)

[[-0.07856742 -1.         -0.04817508 -1.        ]
 [-1.         -0.06337125 -1.         -0.05619329]
 [-0.04817508 -1.         -0.07138946 -1.        ]
 [-1.         -0.05619329 -1.         -0.07714741]] [[-1.         -0.07856742 -1.         -0.01778275]
 [-0.07856742 -1.         -0.04817508 -1.        ]
 [-1.         -0.04817508 -1.         -0.09460383]
 [-0.01778275 -1.         -0.09460383 -1.        ]]


In [ ]:
import numpy as np
from scipy.linalg import eigh, cholesky
from itertools import combinations


def solve_eigen_problem(H_subspace, S_subspace):
    """ Solve the generalized eigenvalue problem and return the smallest eigenvalue and its corresponding eigenvector. """
    eigenvals = None
    for d in range(H_subspace.shape[0]):
        try:
            eigvals, eigvecs = eigh(H_subspace[:d, :d], S_subspace[:d, :d] + 1.125*np.eye(d))
            print(eigvals)
            min_eigval_index = np.argmin(eigvals)
            eigenvals = eigvals[min_eigval_index]
        except:
            pass
    return eigenvals

In [ ]:
solve_eigen_problem(H_subspace, S_subspace)

[-0.62853936]
[-23.06526158   4.56397032]


-23.06526158477751

## Apparently, 2000 circuit depth is too much, let us use variational fast forwarding (VFF)

In [ ]:
iter = 0
def cost_func(params, hamiltonian):
    global qc_copies
    estimator = qiskit.primitives.Estimator(options={"shots": None, "approximation": True})

    global iter
    total_circuits_real = []
    total_observables_real = []
    total_circuits_imag = []
    total_observables_imag = []

    observable_Z = SparsePauliOp("Z" + "I" * (9), coeffs=[1])
    observable_Y = SparsePauliOp("Y" + "I" * (9), coeffs=[1])
    for k in range(0,2*len(krylov_vectors),2):
      ansatz_k = VariationalTimeEvolution(9+1, reps=2, n=int(k/2) if k % 2 == 0 else int(math.floor(k/2)), real=True)
      ansatz_k = ansatz_k.assign_parameters(params)

      trotter = qc_copies[k]

      circuit = QuantumCircuit(10)
      circuit = circuit.compose(trotter, range(1,10))
      circuit = circuit.compose(ansatz_k.inverse(), range(10))

      total_circuits_real.append(circuit)
      total_observables_real.append(observable_Z)
      total_circuits_imag.append(circuit)
      total_observables_imag.append(observable_Y)

    expectation_values_real = estimator.run(total_circuits_real, total_observables_real).result().values
    expectation_values_imag = estimator.run(total_circuits_imag, total_observables_imag).result().values

    cost = 0
    values = []
    for real, imag in zip(expectation_values_real, expectation_values_imag):
      cost += (1/(len(krylov_vectors))) * np.absolute(real + 1.0j*imag)**2
      values.append(np.absolute(real + 1.0j*imag)**2)
      print(np.absolute(real + 1.0j*imag)**2)
    print(f"Iteration: {iter}, Cost: {1-cost}, Real: {real}, Imaginary: {imag}")
    iter += 1
    return 1 - cost

def minimize_func(params):
   global H_eff
   return cost_func(params, H_eff)

In [ ]:
from qiskit_algorithms.optimizers import SPSA, COBYLA, AQGD, NFT, SLSQP, IMFIL, BOBYQA, UMDA

def run_vqe(reps, optimizer_name='COBYLA', maxiter=1000):
  global H_eff
  ansatz = VariationalTimeEvolution(10, reps=2, n=1)

  optimizer = NFT(maxiter=300)
  bounds = [(-np.pi, np.pi)] * ansatz.num_parameters
  initial_parameters = np.random.random(ansatz.num_parameters) * np.pi * (-1)**(np.random.randint(0,100))
  res = optimizer.minimize(minimize_func, initial_parameters, bounds=bounds)

  print("Found Ground State: ", res.fun)
  print("Parameters: ", res.x)
  return res

In [ ]:
res = run_vqe(1)

Streaming output truncated to the last 5000 lines.
0.30317766100719123
0.6871507653237531
0.3023965204885095
0.7900784215093889
0.3757160662720113
0.6941146584744708
0.2600769959548343
0.6896112961196269
0.3901762293596949
0.7111005587831871
0.3169274052527685
0.7104665237187878
0.29148177745842097
Iteration: 308, Cost: 0.48630542899295914, Real: -0.10561497602798653, Imaginary: 0.5294593981572419
0.6173207726934168
0.8230636235816972
0.38941797347622276
0.6054774076401093
0.4845881554703859
0.7794807923094274
0.4301491689961949
0.6520423301499362
0.39221579510905474
0.7007950889080883
0.4596969462657915
0.6751127191800864
0.4520781225232962
0.7540234615737516
0.3498466529899492
Iteration: 309, Cost: 0.4289793992755061, Real: -0.16392438066151455, Imaginary: 0.5683092911563985
0.5871765995694825
0.9936776485855289
0.43916135689735103
0.7829620241937714
0.4660542400445081
0.9551857080482069
0.5033006731161634
0.8191365197591964
0.3904611747684121
0.8449263127323283
0.5257774577037079
0.

### Seems good! Let us run it on the Hardware

In [ ]:
!pip install qiskit_ibm_runtime

In [ ]:
import numpy as np
from qiskit.primitives import Estimator
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.compiler import transpile


from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options


QiskitRuntimeService.save_account(channel="ibm_quantum", token="XXXX", overwrite=True)
service = QiskitRuntimeService(channel="ibm_quantum", instance="rpi-rensselaer/research/rhone")
backend = service.backend("ibm_rensselaer")

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options

options = Options()
options.execution.shots = 10000
options.resilience_level = 2
options.resilience.noise_factors = [1, 1.25, 1.5, 1.75, 2]
options.resilience.extrapolator = "LinearExtrapolator"

from qiskit.transpiler import PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import (
    ALAPScheduleAnalysis,
    PadDynamicalDecoupling,
    ConstrainedReschedule,
)
from qiskit.circuit.library import XGate

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3, approximation_degree=1)
#pm.scheduling = PassManager(
#    [
#        ALAPScheduleAnalysis(target=target),
#        ConstrainedReschedule(target.acquire_alignment, target.pulse_alignment),
#        PadDynamicalDecoupling(
#            target=target, dd_sequence=[XGate(), XGate()], pulse_alignment=target.pulse_alignment
#        ),
#    ]
#)

In [ ]:
options_sampler = Options()
options_sampler.execution.shots = 10000
options_sampler.resilience_level = 1

In [ ]:
optimal_params = [ 1.02352309e+01,  1.06584716e-01, -3.33113117e+00, -9.53576387e-01,
  2.89794858e+00, -2.40705253e+00, -1.62971233e+00,  5.09156019e+00,
 -3.20094487e+00,  4.18585297e-02, -1.79801921e-01, -8.39561668e-02,
 -5.39132540e-01, -8.93474339e-01, -2.04480953e+00, -2.64402491e-01,
  4.76800625e+00, -2.75712961e+00, -2.87596126e+00, -1.28527687e-02,
 -3.11890433e+00, -2.17687114e+00, -9.76097751e-01, -2.89367724e+00,
 -2.99463121e+00, -9.56481727e-01,  7.40728989e+00, -1.46041306e-01,
  5.28697827e+00,  4.13138515e+00,  3.46803855e+00,  1.51415754e+01,
  1.10931158e+00]

In [ ]:
def create_projection_operator_pauli(i, num_ancillary):
    """Create a projection operator for the i-th state in the ancillary register."""
    # Constructs a binary string where only the i-th ancillary qubit is 1 (rest are 0)
    # assuming ordering from most to least significant qubit
    binary_state = str(bin(i)[2:].zfill(5))
    label = "".join(["I" if binary_state[j] != "1" else "Z" for j in range(num_ancillary)])
    return SparsePauliOp.from_list([(label, 1)])


def measure_in_basis(qc, ham, num_ancillary):
    """Measure the expectation value conditioned on the ancillary qubits being in state i."""
    expectation = 0.0
    global options, backend
    with Session(backend=backend) as session:
      estimator = Estimator(options=options)

      circuits = []
      observables = []
      qc = pm.run(qc)
      print(qc.depth())
      for i, (coeff, term) in enumerate(zip(ham.coeffs, ham.paulis)):
        A_i = create_projection_operator_pauli(i, num_ancillary)
        pauli_matrix = SparsePauliOp("I" + str(term), coeffs=[1])
        # Measurement operator combining ancillary projection with system Pauli
        M = A_i.tensor(pauli_matrix)
        M = M.apply_layout(qc.layout)
        observables.append(M)
        circuits.append(qc)

      results = estimator.run(circuits, observables).result()

    expectation = 0.0
    for i, (coeff, result) in enumerate(zip(ham.coeffs, results.values)):
        expectation += coeff * result

    return expectation

def measure_and_decide(qc, num_ancillary):
    global options_sampler
    measurement_operator = create_measurement_operator(num_ancillary)
    idx = 0
    for i in range(4+1,5+4):
      qc.measure(i,idx)
      idx += 1
    print("Running Pass Manager")
    qc = pm.run(qc)
    print(qc.depth)
    print(qc.depth())
    print("Executing on Hardware")
    with Session(backend=backend) as session:
      sampler = Sampler(options=options_sampler)
      result = sampler.run(circuits=[qc]).result()
    print(result.quasi_dists[0])
    return 1 if abs(result.quasi_dists[0][0] - 1) <= 0.1 else -1

In [ ]:
def create_circuit_var(k, ham):
    global R_QC, G_QC, G_inverse_QC, U_QC, r, qT, optimal_params


    # Create the quantum circuit
    num_qubits = 10  # Determine the number of qubits from the state vector size
    if(k % 2 == 0):
      qc = QuantumCircuit(num_qubits, 5)
    else:
      qc = QuantumCircuit(num_qubits)

    var = VariationalTimeEvolution(9+1, reps=2, n=int(k/2) if k % 2 == 0 else int(math.floor(k/2)), real=True)
    qc = qc.compose(var)
    qc = qc.assign_parameters(optimal_params)


    # Applying unitary gates based on the condition of k
    if k % 2 == 0:
        #qc.append(UnitaryGate(qT), range(5))
        decision = measure_and_decide(qc, 5)
        print(decision)
        return decision
    else:
        estimate= measure_in_basis(qc, ham, 5)
        print(estimate)
        return estimate

In [ ]:
last_sign = 1
krylov_entries_simulation = []
for d in range(0,2*len(krylov_vectors)):
  krylov_entries_simulation.append(create_circuit_var(d, H_eff))

Running Pass Manager
<bound method QuantumCircuit.depth of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd3580744c0>>
324
Executing on Hardware


<ipython-input-98-1bf9e74bf765>:51: DeprecationWarning: The Sampler and Estimator V1 primitives have been deprecated as of qiskit-ibm-runtime 0.23.0 and will be removed no sooner than 3 months after the release date. Please use the V2 Primitives. See the `V2 migration guide <https://docs.quantum.ibm.com/api/migration-guides/v2-primitives>`_. for more details
  sampler = Sampler(options=options_sampler)


{0: 0.104981192816538, 1: 0.054271343362124, 2: 0.055815701011881, 3: 0.061964304083852, 4: 0.088690144071583, 5: 0.047100791374325, 6: 0.058246155408032, 7: 0.049524354671959, 8: 0.066089727393169, 9: 0.068038039203839, 10: 0.080927520036135, 11: 0.06295080634142, 12: 0.03746626753351, 13: 0.046184049087031, 14: 0.071437658620798, 15: 0.046311944983805}
-1


<ipython-input-98-1bf9e74bf765>:15: DeprecationWarning: The Sampler and Estimator V1 primitives have been deprecated as of qiskit-ibm-runtime 0.23.0 and will be removed no sooner than 3 months after the release date. Please use the V2 Primitives. See the `V2 migration guide <https://docs.quantum.ibm.com/api/migration-guides/v2-primitives>`_. for more details
  estimator = Estimator(options=options)


327
(-0.009415760381557331+0j)
Running Pass Manager
<bound method QuantumCircuit.depth of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd351ea8250>>
666
Executing on Hardware
{0: 0.058672000279127, 1: 0.05730551938144, 2: 0.057094660978035, 3: 0.062039820078739, 4: 0.06491140293426, 5: 0.061004057866494, 6: 0.056507659112744, 7: 0.06265527839691, 8: 0.065720747559944, 9: 0.066374916132729, 10: 0.069577147252908, 11: 0.064798687464123, 12: 0.059771410537744, 13: 0.067549162195145, 14: 0.063012209207334, 15: 0.063005320622323}
-1
617
(0.02787896399918199+0j)
Running Pass Manager
<bound method QuantumCircuit.depth of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd3594773d0>>
650
Executing on Hardware
{0: 0.065520464816303, 1: 0.058735516424735, 2: 0.060333291184696, 3: 0.068552122177661, 4: 0.065573358363535, 5: 0.066128262343927, 6: 0.067398103703395, 7: 0.064035605471427, 8: 0.059308379622427, 9: 0.052899491682067, 10: 0.059520667984269, 11: 0.054241015131264

### Brute Force Compile with Low-Fidelity, how well does it perform?

In [ ]:
krylov_entries_simulation = [-1, 0.0849163, -1, -0.008295, -1, -0.021309, -1, 0.04697]

In [ ]:
H_subspace, S_subspace = generate_subspace_matricies(krylov_entries_simulation, 4)

In [ ]:
import numpy as np
from scipy.linalg import eigh, cholesky
from itertools import combinations


def solve_eigen_problem(H_subspace, S_subspace):
    """ Solve the generalized eigenvalue problem and return the smallest eigenvalue and its corresponding eigenvector. """
    eigenvals = None
    for d in range(H_subspace.shape[0]):
        try:
            eigvals, eigvecs = eigh(H_subspace[:d, :d], S_subspace[:d, :d] + 1.088*np.eye(d))
            print(eigvals)
            min_eigval_index = np.argmin(eigvals)
            eigenvals = eigvals[min_eigval_index]
        except:
            pass
    return eigenvals

In [ ]:
solve_eigen_problem(H_subspace, S_subspace)

[0.96495795]
[ -5.36016375 348.04524079]


-5.360163745641313

#### Well, it is something, let us try better with our variational circuit


### Variational Fast Forwarding

For ease, I just ran it above as well replacing the circuit construction part with our variational form, here is the data:

In [ ]:
values = [-1, -0.0094157, -1, 0.2787896, -1, 0.0114861, -1, -0.0122492, -1, 0.00553, -1, -0.000655, -1, 0.016885, -1, -0.017789, -1, -0.0060220169949944755, -1, -0.019586862501721453, -1, -0.02266568633767947, -1, -0.01696326428758073, -1, -0.0021185411389147346]

In [ ]:
H_subspace, S_subspace = generate_subspace_matricies(values, 4)

In [ ]:
import numpy as np
from scipy.linalg import eigh, cholesky
from itertools import combinations


def solve_eigen_problem(H_subspace, S_subspace):
    """ Solve the generalized eigenvalue problem and return the smallest eigenvalue and its corresponding eigenvector. """
    eigenvals = None
    k = 0
    for d in range(H_subspace.shape[0]):
        try:
            print( S_subspace[k:d, k:d])
            eigvals, eigvecs = eigh(H_subspace[k:d, k:d], S_subspace[k:d, k:d] + 1.05*np.eye(d))
            print(eigvals)
            min_eigval_index = np.argmin(eigvals)
            eigenvals = eigvals[min_eigval_index]
        except:
            pass
    return eigenvals

### Notice, to prevent loss of positive definiteness, I had to normalize the diagonal by 1.05, so we really are uncertain about energy by 1.05, meaning we really only know for we converged to E = -22 for sure.

In [ ]:
solve_eigen_problem(H_subspace, S_subspace)

[]
[[-1.]]
[-0.188314]
[[-1.        -0.0094157]
 [-0.0094157 -1.       ]]
[-23.99743856  17.29146887]
[[-1.         -0.0094157  -1.        ]
 [-0.0094157  -1.          0.13468695]
 [-1.          0.13468695 -1.        ]]


-23.997438557474794